2. Cargamos los datos prosesados en la tabla delta

In [0]:
# Cargar esquema de la tabla Delta existente
tabla_delta = "aprendisaje.default.servicio_restaurantes"
ruta_processed = "dbfs:/mnt/ingest_data/processed/"

try:
    esquema_delta = spark.table(tabla_delta).schema
except Exception as e:
    print(f"Error al cargar el esquema de la tabla Delta: {e}")
    raise

# Listar archivos procesados
archivos_procesados = [f.path for f in dbutils.fs.ls(ruta_processed)]

for archivo in archivos_procesados:
    # Leer archivo procesado
    df_spark = spark.read.format("parquet").load(archivo)
    
    # Validar columnas
    columnas_comunes = [col for col in esquema_delta.fieldNames() if col in df_spark.columns]
    
    # Validar tipos de datos y convertir si es necesario
    df_alineado = df_spark.select(
        *[
            df_spark[col].cast(esquema_delta[col].dataType).alias(col)
            for col in columnas_comunes
        ]
    )
    
    # Escribir datos en la tabla Delta
    try:
        df_alineado.write.format("delta").mode("append").saveAsTable(tabla_delta)
        print(f"Datos cargados desde: {archivo}")
    except Exception as e:
        print(f"Error al cargar datos desde {archivo}: {e}")

print("Todos los datos se han cargado exitosamente en la tabla Delta.")


Datos cargados desde: dbfs:/mnt/ingest_data/processed/etl_datos_de_prueba.parquet
Todos los datos se han cargado exitosamente en la tabla Delta.
